In [ ]:
%pip install neuronpedia

from dotenv import load_dotenv

# Load NEURONPEDIA_API_KEY from .env file
load_dotenv()

### [recommended] Use context-based API key management
# import neuronpedia

# Option 1: Context manager (recommended for temporary scope)
# with neuronpedia.api_key("your-api-key-here"):
#     # Your code here

# Option 2: Global context setting (good for notebook-wide usage)
# neuronpedia.set_api_key("your-api-key-here")

### [less safe] Set Neuronpedia API key manually (get your key from neuronpedia.org/account)
# import os
# os.environ["NEURONPEDIA_API_KEY"] = "YOUR_KEY_HERE"

### Step 1) Create or ensure the model exists on Neuronpedia


In [ ]:
# Check if the model exists by going to neuronpedia.org and looking at the list of models.
# You can also go to https://neuronpedia.org/[model_id] to see if it exists, in case it is an unlisted model.
#
# If the model doesn't exist, create it:

from neuronpedia.np_model import Model

# IMPORTANT: Change the id, number of layers, and display name (optional).
MODEL_ID = "pup-xl"  # change this to the ID of your model
NUMBER_OF_LAYERS = 3  # change this to the number of layers in your model
MODEL_DISPLAY_NAME = (
    "Puppy Model XL"  # optional for display purposes, will default to the model ID
)

model = Model.new(
    id=MODEL_ID,
    layers=NUMBER_OF_LAYERS,
    display_name=MODEL_DISPLAY_NAME,
    # url="https://creator-of-my-test-model.com/my-test-model", # optional, for display purposes
)

print("Created model:")
print(model)


### Step 2) Create your source set on Neuronpedia

Before uploading features, you must create a **source set**, which is a collection of **sources**.

**Each source is a set of features for one layer of a model.** _Example: one SAE, which can contain thousands of features, is one source._

**Sources are named as `[layer_number]-[source_set_name]`.** \_Example: "20-gemmascope-res-16k" is layer 20, and its source set name is gemmascope-res-16k.

**When you create a source set, it automatically creates a source for each layer of the model.** \_Example: If you create a source set called "puppyscope-res-16k" for a 3 layer "pup-model-3" model, it will create 3 sources: "0-puppyscope-res-16k", "1-puppyscope-res-16k", and "2-puppyscope-res-16k".

**How to Name Your Source Set**
There are no strict rules other than alphanumeric and hyphens. But try to keep it under 32 characters, and include maybe the author or abbreviation of author, and hook used.

Let's create a source set:


In [ ]:
from neuronpedia.np_source_set import SourceSet

# Note that we use the same model ID as above. Since that model ID has 3 layers, this command will create 3 sources: "0-puppyscope-res-2k", "1-puppyscope-res-2k", and "2-puppyscope-res-2k".
SOURCE_SET_NAME = "puppyscope-res-2k"  # change this to the name of your source set

source_set = SourceSet.new(
    model_id=MODEL_ID,
    name=SOURCE_SET_NAME,
)

print("Created source set:")
print(source_set)

# If we need get which sources were created in the future, we can fetch the source set from Neuronpedia and print it:
source_set = SourceSet.get(model_id=MODEL_ID, name=SOURCE_SET_NAME)

print("Fetched source set:")
print(source_set)


### Step 3) Upload features (and activations + explanations) to your sources

**After creating the source set, you can upload features (and activations + explanations) to it, by referencing its source.** \_Example: For the source set above, you upload the to layer 1 by first getting the layer 1 source, then calling the `source.upload_batch()` method with the feature details for each feature.

Only the creator of the source set can upload features to it, so be sure to set the correct Neuronpedia API_KEY.

#### Note about activations + quantiles:

- Quantiles are optional, but if you want to set them, you should add the the quantileMax, quantileMin, and quantileFraction values.
- The quantileMax is the maximum activation value of all activations in this quantile.
- The quantileMin is the minimum activation value of all activations in this quantile.
- The quantileFraction is the fraction of texts that activated in this quantile. For example, if you have 4 even quantiles, then quantileFraction will be 0.25 for each of them.
- If two activations have the same quantileMax, quantileMin, and quantileFraction, they show up in the same section in the activations list in the UI.
- Neuronpedia's UI will automatically sort from sections with the highest quantileMax to sections to the lowest quantileMax.


In [ ]:
from neuronpedia.np_feature import Feature
from neuronpedia.np_explanation import Explanation
from neuronpedia.np_activation import Activation
from neuronpedia.np_feature import Logit

# Let's upload two example features in layer 1, each with an explanation and 2 activations.

# First, get the source for layer 1.
source = source_set.get_source_for_layer_number(1)
print("Got source_id for layer 1:", source)

# Then, make two example features to upload.

# ========== feature one - dog stuff ==========

# Create the base feature to add explanations and activations to.
new_feature_1 = Feature(
    modelId=MODEL_ID,
    source=source.id,
    index=0,  # feature at index 0
    density=0.03,  # [optional] activations density - from zero to one.
    top_logits=[Logit("dog", 0.9), Logit("cat", 0.7)],  # optional - top logits
    bottom_logits=[
        Logit("car", -1.1),
        Logit("house", -0.9),
    ],  # optional - bottom logits
)

# Add an explanation and 2 activations to the first feature.
new_feature_1_activations = [
    Activation(
        modelId=new_feature_1.modelId,
        source=new_feature_1.source,
        index=new_feature_1.index,
        tokens=["my", " border", " collie"],
        values=[0, 2.5, 5.3],
        quantileMax=6,  # optional - this is the max value in this quantile bin. bins are defined by the quantileMin and quantileMax. if two activations have the same quantileMin and quantileMax, they show up in the same bin.
        quantileMin=5,  # optional - this is the min value in this quantile bin
        quantileFraction=0.1,  # optional - this is the fraction of values in this quantile bin
    ),
    Activation(
        modelId=new_feature_1.modelId,
        source=new_feature_1.source,
        index=new_feature_1.index,
        tokens=["the", " golden", " retriever"],
        values=[0.2, 3.1, 4.8],
        quantileMax=5,
        quantileMin=4,
        quantileFraction=0.05,
    ),
]
new_feature_1.activations = new_feature_1_activations

new_feature_1_explanation = Explanation(
    modelId=new_feature_1.modelId,
    source=new_feature_1.source,
    index=new_feature_1.index,
    text="magic dog",
    # method="oai_token-act-pair"  # optional explanation method. must be supported by Neuronpedia.
    # explainer_model="gpt-4o"     # optional explainer model. must be supported by Neuronpedia.
)
new_feature_1.explanations = [new_feature_1_explanation]

# ========== feature two - cat stuff ==========

new_feature_2 = Feature(
    modelId=MODEL_ID,
    source=source.id,
    index=1,  # feature at index 1
    density=0.05,  # [optional] activations density - from zero to one.
)

# Add an explanation and 2 activations to the second feature.
new_feature_2_activations = [
    Activation(
        modelId=new_feature_2.modelId,
        source=new_feature_2.source,
        index=new_feature_2.index,
        tokens=["my", " tabby", " cat"],
        values=[0, 1.5, 3.2],
    ),
    Activation(
        modelId=new_feature_2.modelId,
        source=new_feature_2.source,
        index=new_feature_2.index,
        tokens=["the", " persian", " cat"],
        values=[0.1, 2.4, 4.1],
    ),
]
new_feature_2.activations = new_feature_2_activations

new_feature_2_explanation = Explanation(
    modelId=new_feature_2.modelId,
    source=new_feature_2.source,
    index=new_feature_2.index,
    text="magic cat",
)
new_feature_2.explanations = [new_feature_2_explanation]

# ========== batch upload them ==========

# Finally, upload the features.
source.upload_batch(
    features=[new_feature_1, new_feature_2],
)

print("Features uploaded successfully!")


### Finally - View the Features

These features are accessible by URL directly. They are not listed on the home page or in default dropdowns.

You can also view the source and source set pages to browse them.


In [6]:
# view feature #1 in the browser
new_feature_1.open_in_browser()

In [17]:
# view feature #1 in the browser
new_feature_2.open_in_browser()

In [20]:
# view the source in the browser
source.open_in_browser()